# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846714780244                   -0.71    4.5         
  2   -7.852180852320       -2.26       -1.54    1.0   18.1ms
  3   -7.852615177583       -3.36       -2.57    2.0   20.2ms
  4   -7.852645905854       -4.51       -2.96    2.2   22.3ms
  5   -7.852646566003       -6.18       -3.38    1.0   17.3ms
  6   -7.852646676873       -6.96       -4.00    1.5   19.3ms
  7   -7.852646686414       -8.02       -5.68    1.5   60.6ms
  8   -7.852646686729       -9.50       -6.21    3.2   26.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846858316488                   -0.70           4.5         
  2   -7.852526347074       -2.25       -1.63   0.80    2.0    207ms
  3   -7.852636345317       -3.96       -2.72   0.80    1.0   16.1ms
  4   -7.852646600177       -4.99       -3.26   0.80    2.2   21.2ms
  5   -7.852646682314       -7.09       -4.13   0.80    1.0   15.8ms
  6   -7.852646686359       -8.39       -4.92   0.80    1.2   17.4ms
  7   -7.852646686720       -9.44       -5.87   0.80    2.0   20.2ms
  8   -7.852646686730      -11.00       -7.02   0.80    2.2   21.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.392999e+01     3.200008e+00
 * time: 0.046170949935913086
     1     1.604474e+00     1.975501e+00
 * time: 0.21927499771118164
     2    -1.478369e+00     2.357089e+00
 * time: 0.23728203773498535
     3    -3.719226e+00     2.206651e+00
 * time: 0.26306700706481934
     4    -5.172344e+00     2.113847e+00
 * time: 0.28878092765808105
     5    -6.719548e+00     1.455641e+00
 * time: 0.3146169185638428
     6    -6.888665e+00     1.783924e+00
 * time: 0.33242106437683105
     7    -7.449353e+00     1.324456e+00
 * time: 0.35024189949035645
     8    -7.516555e+00     1.510260e+00
 * time: 0.3681049346923828
     9    -7.673405e+00     1.039300e+00
 * time: 0.38605189323425293
    10    -7.723542e+00     8.209456e-01
 * time: 0.40405797958374023
    11    -7.783759e+00     1.958426e-01
 * time: 0.4219400882720947
    12    -7.805167e+00     1.119282e-01
 * time: 0.43982887268066406
    13    -7.833288e+00     1.185862e-01
 * time: 0

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846813962279                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645890039                   -1.64         
  2   -7.852646686730       -6.10       -3.70    1.57s
  3   -7.852646686730      -13.31       -7.24    122ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.585690510645978e-7
|ρ_newton - ρ_scfv| = 1.5297640832487523e-7
|ρ_newton - ρ_dm|   = 6.398055870980295e-10
